In [1]:
import os
import pandas as pd
import glob
from pandas.core.accessor import delegate_names
from pandas.core.frame import DataFrame
import requests

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


def split_csv(total_csv):
    rows = pd.read_csv(total_csv,chunksize=5000)
    file_count = 0
    for i, chuck in enumerate(rows):
        chuck.to_csv('./result/out{}.csv'.format(i),encoding='utf-8-sig',index=False)
        file_count = file_count+1
    return file_count


def merge_csv():
  input_file = r'./result/'
  output_file = r'./result/result'

  allFile_list = glob.glob(os.path.join(input_file, 'fout*'))
  allFile_list.sort()
  for file in allFile_list:
      output_file = output_file + file[-5]
  output_file = output_file +'.csv'

  all_Data = []
  for file in allFile_list:
    records = pd.read_csv(file).to_dict('records')
    all_Data.extend(records)

  DataFrame(all_Data).to_csv(output_file,encoding='utf-8-sig',index=False)
  print("결과물 파일 : " + output_file)




In [6]:
!rm -r result


In [3]:
import requests
import json
from bs4 import BeautifulSoup
import re

mint_query_template = '''
{
  mints(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s  }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

mint_query_first = '''
{
  mints(first: 1, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''


swap_query_template = '''
{
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
'''


burn_query_template = '''
{
  burns(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''


def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/30f67c1ef7d30ea143e6aab98a97703b/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

def call_theGraph_mint(pair_id):
    mint_array = []
    timestamp = 0
    try:
      while(True):
        query = mint_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['mints']) < 1000):
          mint_array.extend(result['data']['mints'])
          break

        mint_array.extend(result['data']['mints'])
        timestamp = result['data']['mints'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return mint_array

def call_theGraph_swap(pair_id):
    swap_array = []
    timestamp = 0
    try:
      while(True):
        query = swap_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['swaps']) < 1000):
          swap_array.extend(result['data']['swaps'])
          break

        swap_array.extend(result['data']['swaps'])
        timestamp = result['data']['swaps'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return swap_array

def call_theGraph_burn(pair_id):
    burn_array = []
    timestamp = 0
    try:
      while(True):
        query = burn_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['burns']) < 1000):
          burn_array.extend(result['data']['burns'])
          break

        burn_array.extend(result['data']['burns'])
        timestamp = result['data']['burns'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_burn')
      print(e)

    return burn_array



def get_creatorAddress(pair_id,token_id):
    repos_url = 'https://api.ethplorer.io/getAddressInfo/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N'
    response = requests.get(repos_url).text
    repos = json.loads(response)

    try:
        creator_address = repos['contractInfo']['creatorAddress']
        if(creator_address == None):
          raise ValueError
    except:
         url = 'https://etherscan.io/address/'+token_id
         try:
             response = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})
             page_soup = BeautifulSoup(response.text, "html.parser")
             Transfers_info_table_1 = str(page_soup.find("a", {"class": "hash-tag text-truncate"}))
             creator_address = re.sub('<.+?>', '', Transfers_info_table_1, 0).strip()
             if(creator_address == 'None'):
                query = mint_query_first % pair_id
                response = run_query(query)
                creator_address = response['data']['mints'][0]['to']
         except Exception as e:
              print(e)
              creator_address = 'Fail to get Creator Address'

    # if creator_address in proxy_contracts:
    #     query = mint_query_first % pair_id
    #     response = run_query(query)
    #     creator_address = response['data']['mints'][0]['to']

    return creator_address

def get_holders(token_id):
    repos_url = 'https://api.ethplorer.io/getTopTokenHolders/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N&limit=100'
    response = requests.get(repos_url)
    if(response.status_code == 400):
        return []
    repos = json.loads(response.text)
    return repos['holders']

In [4]:
from decimal import Decimal
from math import sqrt

def get_initial_Liquidity(token0_symbol,mint_data_transaction):
  if(token0_symbol == 'WETH'):
      initial_Liquidity_ETH = mint_data_transaction[0]['amount0']
      initial_Liquidity_token = mint_data_transaction[0]['amount1']
  else:
      initial_Liquidity_ETH = mint_data_transaction[0]['amount1']
      initial_Liquidity_token = mint_data_transaction[0]['amount0']

  return initial_Liquidity_ETH,initial_Liquidity_token

def get_initial_Liquidity_token(mint_data_transaction,index):
  if(index == 1):
    return Decimal(mint_data_transaction[0]['amount1'])
  else:
    return Decimal(mint_data_transaction[0]['amount0'])

def get_mint_mean_period(mint_data_transaction,initial_timestamp):
    count = len(mint_data_transaction)
    if(count == 0):
      return 0
    mint_time_add = 0
    for transaction in mint_data_transaction:
      mint_time_add = mint_time_add + int(transaction['timestamp']) - initial_timestamp
    return mint_time_add / count

def get_swap_mean_period(swap_data_transaction,initial_timestamp):
    count = len(swap_data_transaction)
    if(count == 0):
      return 0
    swap_time_add = 0
    for transaction in swap_data_transaction:
      swap_time_add = swap_time_add +  int(transaction['timestamp']) - initial_timestamp
    return swap_time_add / count

def get_burn_mean_period(burn_data_transaction,initial_timestamp):
    count = len(burn_data_transaction)
    if(count == 0):
      return 0
    burn_time_add = 0
    for transaction in burn_data_transaction:
      burn_time_add = burn_time_add + int(transaction['timestamp']) - initial_timestamp
    return burn_time_add / count

def swap_IO_rate(swap_data_transaction,index):
  swapIn = 0
  swapOut = 0
  if(index == 1):
    for data in swap_data_transaction:
      if(data['amount0In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn + 1
  else:
    for data in swap_data_transaction:
      if(data['amount1In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn +1

  return swapIn,swapOut

def get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction):
  #mint_data_transaction
  swap_len = len(swap_data_transaction)
  burn_len = len(burn_data_transaction)
  #Case 1 Swap / Burn
  if(swap_len == 0 and burn_len == 0):
    return int(mint_data_transaction[-1]['timestamp'])
  #Case 2 Swap_transaction
  if(swap_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']))
  #Case 3 Burn Transaction
  if(burn_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  #Case 4
  return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))

def get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut):
  if(swap_data_transaction[j][eth_amountIn] == '0'):
    return Decimal(swap_data_transaction[j][eth_amountOut]) * (-1)
  else:
    return Decimal(swap_data_transaction[j][eth_amountIn])

def get_swap_token(swap_data_transaction,j,index):
  if(index == 1):
    swap_amount = Decimal(swap_data_transaction[j]['amount1In'])
    swap_amount = Decimal(swap_amount) - Decimal(swap_data_transaction[j]['amount1Out'])
  else:
    swap_amount = Decimal(swap_data_transaction[j]['amount0In'])
    swap_amount = Decimal(swap_amount) - Decimal(swap_data_transaction[j]['amount0Out'])

  return swap_amount

def get_timestamp(data_transaction,index):
  try:
    return data_transaction[index]['timestamp']
  except:
    return '99999999999'

def check_rugpull(before_transaction_Eth, current_Eth):
  if ( abs(Decimal(current_Eth) / Decimal(before_transaction_Eth)) <= 0.01 ):
    if( Decimal(before_transaction_Eth) < 0 or Decimal(current_Eth) < 0):
      return False
    else:
      return True
  else:
    return False

def is_MEV(initial_Liquidity_token, swapIn_token):
  if(swapIn_token > initial_Liquidity_token * 5):
    return False
  else:
    return True

def get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,index):
    if(index == 1):
      eth_amount = 'amount0'
      eth_amountIn = 'amount0In'
      eth_amountOut = 'amount0Out'
    else:
      eth_amount = 'amount1'
      eth_amountIn = 'amount1In'
      eth_amountOut = 'amount1Out'


    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)



    current_Liquidity_Eth = Decimal(mint_data_transaction[0][eth_amount])
    initial_Liquidity_token = get_initial_Liquidity_token(mint_data_transaction,index)
    i,j,k = 1,0,0   #mint,swap,burn

    while True:
      try:
        next_timestamp = min(get_timestamp(mint_data_transaction,i),get_timestamp(burn_data_transaction,k))
        #swap
        while(get_timestamp(swap_data_transaction,j) <= next_timestamp ):
          if(get_timestamp(swap_data_transaction,j) == '99999999999'):
            break


          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth + get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut)
          #print("swap {before : %s swap_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))

          if( check_rugpull(before_transaction_Eth,current_Liquidity_Eth) ):
              if( is_MEV(initial_Liquidity_token,get_swap_token(swap_data_transaction,j,index)) == False ):
                print("swap rugpull : initial token = %s / before Eth = %s / after Eth = %s swapIn_token_amount = %s"%(initial_Liquidity_token,str(before_transaction_Eth),str(current_Liquidity_Eth),get_swap_token(swap_data_transaction,j,index)))
                return get_timestamp(swap_data_transaction,j), Decimal(current_Liquidity_Eth / before_transaction_Eth) -1, True, before_transaction_Eth,current_Liquidity_Eth,'swap'
          j = j+1


        if(next_timestamp == get_timestamp(mint_data_transaction,i)):
          if(next_timestamp == '99999999999'):
            try:

                if(swap_count == 0 and burn_count == 0):
                    return mint_data_transaction[-1]['timestamp'],0, False, 0,0,''

                if(swap_count == 0):
                    return max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']),0,False, 0,0,''

                if(burn_count == 0):
                    return max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']),0,False, 0,0,''

                return max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']),0,False, 0,0,''
            except:
              return 'Error occur',100.0,False,1,1
          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth + Decimal(mint_data_transaction[i][eth_amount])
          #print("mint {before : %s burn_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))
          i = i+1


        else:
          before_transaction_Eth = current_Liquidity_Eth
          current_Liquidity_Eth = current_Liquidity_Eth - Decimal(burn_data_transaction[k][eth_amount])
          #print("burn {before : %s burn_amount : %s"%(str(before_transaction_Eth),str(current_Liquidity_Eth-before_transaction_Eth)))
          if(check_rugpull(before_transaction_Eth,current_Liquidity_Eth)):
            return get_timestamp(burn_data_transaction,k), Decimal(current_Liquidity_Eth / before_transaction_Eth) -1, True, before_transaction_Eth,current_Liquidity_Eth,'burn'
          k = k+1
      except Exception as e:
        print(e)
        print('Critical Error Occur')
        return '1',0,False,1,1,'Error'


def token_index(data):
    if(data['token0.name'] == 'Wrapped Ether'):
        return 1
    else:
        return 0


def calc_LP_distribution(holders):
    count = 0
    for holder in holders:
        if(holder['share'] < 0.01 ):
            break
        count = count +1

    LP_avg = 100 / count
    var = 0
    for i in range(count):
        var = var + (holders[i]['share'] - LP_avg) ** 2

    LP_stdev = sqrt(var)

    return LP_avg,LP_stdev



Locker_address = [
'0x663a5c229c09b049e36dcc11a9b0d4a8eb9db214',   #locker
'0xe2fe530c047f2d85298b07d9333c05737f1435fb',   #locker
'0x000000000000000000000000000000000000dead' ]


def get_Lock_ratio(holders):
    for holder in holders:
        if(holder['address'] in Locker_address):
          return holder['share']
    return 0


Burn_address = [
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000001',
  '0x0000000000000000000000000000000000000002',
  '0x0000000000000000000000000000000000000003',
  '0x0000000000000000000000000000000000000004',
  '0x0000000000000000000000000000000000000005',
  '0x0000000000000000000000000000000000000006',
  '0x0000000000000000000000000000000000000007',
  '0x0000000000000000000000000000000000000008',
  '0x0000000000000000000000000000000000000009',
  '0x000000000000000000000000000000000000000a',
  '0x000000000000000000000000000000000000000b',
  '0x000000000000000000000000000000000000000c',
  '0x000000000000000000000000000000000000000d',
  '0x000000000000000000000000000000000000000e',
  '0x000000000000000000000000000000000000000f',
  '0x000000000000000000000000000000000000dead',
  '0x000000000000000000000000000000000000DEAD'
]


def get_burn_ratio(holders):
  for holder in holders:
    if(holder['address'] in Burn_address):
      return holder['share']

  return 0


def get_creator_ratio(holders,creator_address):
  for holder in holders:
    if(holder['address'] == creator_address):
      return holder['share']

  return 0




In [ ]:
from pandas.core.frame import DataFrame
import pandas as pd
import time
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import Process
import datetime

def switch_file(file_name):
    global datas
    datas = pd.read_csv(file_name).to_dict('records')

def get_feature(data):

    pair_address = data['id']
    token_address = data['token00.id']


    mint_data_transaction = call_theGraph_mint(pair_address)
    swap_data_transaction = call_theGraph_swap(pair_address)
    burn_data_transaction = call_theGraph_burn(pair_address)

    #initial_Liquidity
    initial_Liquidity_Eth , initial_Liquidity_Token = get_initial_Liquidity(data['token0.symbol'],mint_data_transaction)


    mint_count = len(mint_data_transaction)
    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)

    # Mint/Burn/Swap
    initial_timestamp = int(mint_data_transaction[0]['timestamp'])
    last_timestamp = get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction)
    active_period = last_timestamp - initial_timestamp
    mint_mean_period = int(get_mint_mean_period(mint_data_transaction,initial_timestamp))
    swap_mean_period = int(get_swap_mean_period(swap_data_transaction,initial_timestamp))
    burn_mean_period = int(get_burn_mean_period(burn_data_transaction,initial_timestamp))

    #SwapIn/SwapOut
    swapIn,swapOut = swap_IO_rate(swap_data_transaction,token_index(data))


    rugpull_timestamp, rugpull_change, is_rugpull, before_rugpull_Eth, after_rugpull_Eth,rugpull_method = get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,token_index(data))

    #rugpull
    rugpull_proceeding_time = int(rugpull_timestamp) - int(initial_timestamp)
    if(is_rugpull == False):
        rugpull_proceeding_time = 0
        rugpull_method = ''
        rugpull_timestamp = '0'
        rugpull_change = ''

    creater_Address = get_creatorAddress(pair_address, token_address)
    holder_Address = get_holders(token_address)
    lp_Avg, lp_Std = calc_LP_distribution(holder_Address)
    lp_lockRatio = get_Lock_ratio(holder_Address)
    creater_holdingRatio = get_creator_ratio(holder_Address, creater_Address)
    burnRatio = get_burn_ratio(holder_Address)
    initial_Liquidity_Token = float(initial_Liquidity_Token)
    fswapIn = float(swapIn)
    isMev = is_MEV(initial_Liquidity_Token, fswapIn)

    data['initial_Liquidity_Eth'] = initial_Liquidity_Eth
    data['initial_Liquidity_Token'] = initial_Liquidity_Token
    data['last_transaction_timestamp'] = last_timestamp
    data['last_transaction_date'] = datetime.datetime.fromtimestamp(int(last_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    data['mint_count'] = mint_count
    data['swap_count'] = swap_count
    data['burn_count'] = burn_count
    data['mint_mean_period'] = mint_mean_period
    data['swap_mean_period'] = swap_mean_period
    data['burn_mean_period'] = burn_mean_period
    data['swapIn'] = swapIn
    data['swapOut'] = swapOut
    data['token_burn_ratio'] = burnRatio
    data['lp_avg'] = lp_Avg
    data['lp_std'] = lp_Std
    data['creator_holding_ratio'] = creater_holdingRatio
    data['lp_lock_ratio'] = lp_lockRatio
    data['is_MEV'] = isMev
    data['active_period'] = active_period
    data['rugpull_method'] = rugpull_method
    data['rugpull_timestamp'] = rugpull_timestamp
    data['rugpull_timestamp_date'] = datetime.datetime.fromtimestamp(int(rugpull_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    data['before_rugpull_Eth'] = before_rugpull_Eth
    data['after_rugpull_Eth'] = after_rugpull_Eth
    data['rugpull_change'] = rugpull_change
    data['rugpull_proceeding_hour'] = str(rugpull_proceeding_time / 3600) + 'h'
    data['is_rugpull'] = is_rugpull



    return data



# if __name__=='__main__':
#     createFolder('./result')
#     file_name = 'zero.csv'
#     file_count = split_csv(file_name)
#     out_list = []
#     out_list = list(input('입력(공백단위) : ').split())

#     for i in out_list:         #하나의 파일 단위로 Creator Address 불러오고, 해당 초기 유동성풀 이더값 구해온다.
#         file_name = './result/out{}.csv'.format(i)
#         switch_file(file_name)
#         datas_len = len(datas)
#         try:
#             p = Pool(1)
#             count = 0
#             result = []
#             for ret in p.imap(get_feature,datas):
#                 count = count+1
#                 result.append(ret)
#                 if(count % 200 == 0):
#                     print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
#             p.close()
#             p.join()
#         except Exception as e:
#             print(e)
#         print('===================================   finish    =========================================')
#         time.sleep(5)

#         df = pd.DataFrame(result)
#         file_name = './result/fout{}.csv'.format(i)
#         df.to_csv(file_name,encoding='utf-8-sig',index=False)
#         print(file_name + ' complete')
#     merge_csv()



# import time
# import pandas as pd
# from multiprocessing import Pool

# if __name__=='__main__':
#     createFolder('./result')
#     file_name = 'split_0.csv'
#     file_count = split_csv(file_name)
#     out_list = []
#     out_list = list(input('입력(공백단위) : ').split())

#     for i in out_list:  # Process each input file
#         file_name = './result/out{}.csv'.format(i)
#         switch_file(file_name)
#         datas_len = len(datas)

#         p = Pool(1)
#         count = 0
#         result = []

#         try:
#             for data in datas:
#                 try:
#                     ret = p.apply_async(get_feature, (data,)).get()
#                     result.append(ret)
#                     count += 1
#                     if count % 200 == 0:
#                         print("Process Rate : {}/{} {}%".format(count, datas_len, int((count/datas_len)*100)))
#                 except Exception as e:
#                     print(f"Error processing data point: {data}, error: {e}")

#             p.close()
#             p.join()
#         except Exception as e:
#             print(f"Error in processing pool: {e}")

#         print('===================================   finish    =========================================')
#         time.sleep(1)

#         df = pd.DataFrame(result)
#         file_name = './result/fout{}.csv'.format(i)
#         df.to_csv(file_name, encoding='utf-8-sig', index=False)
#         print(file_name + ' complete')

#     merge_csv()


import time
import pandas as pd

def main():
    createFolder('./result')
    file_name = 'SMALLsplit_1.csv'
    file_count = split_csv(file_name)
    out_list = []
    out_list = list(input('How many splits) : ').split())

    for i in out_list:  # Process each input file
        file_name = './result/out{}.csv'.format(i)
        switch_file(file_name)
        datas_len = len(datas)

        count = 0
        result = []

        try:
            for data in datas:
                try:
                    ret = get_feature(data)  # Directly call the function
                    result.append(ret)
                    count += 1
                    if count % 200 == 0:
                        print("Process Rate : {}/{} {}%".format(count, datas_len, int((count/datas_len)*100)))
                except Exception as e:
                    print(f"Error processing data point: {data}, error: {e}")

        except Exception as e:
            print(f"Error in processing: {e}")

        print('===================================   finish    =========================================')
        time.sleep(1)

        df = pd.DataFrame(result)
        file_name = './result/fout{}.csv'.format(i)
        df.to_csv(file_name, encoding='utf-8-sig', index=False)
        print(file_name + ' complete')

    merge_csv()

if __name__ == '__main__':
    main()



How many splits) : 0
swap rugpull : initial token = 800000000 / before Eth = 2.582563906653630957 / after Eth = 1.60645781667E-7 swapIn_token_amount = 10000000000000000
swap rugpull : initial token = 985000000 / before Eth = 9.161014702381636085 / after Eth = 0.000005922317690071 swapIn_token_amount = 168276000000000
swap rugpull : initial token = 29517000 / before Eth = 11.389191842608462168 / after Eth = 2.97042172104E-7 swapIn_token_amount = 999999999999999
swap rugpull : initial token = 100000000 / before Eth = 10.954906214313802756 / after Eth = 1.23E-16 swapIn_token_amount = 1000000000000000000000000
Error processing data point: {'createdAtBlockNumber': 19934788, 'createdAtTimestamp': '2024-5-23', 'id': '0xd09e2617ba70599f93cc8dc12f02697dd863eead', 'reserve0': 0.0, 'reserve1': 0.0, 'reserveETH': 0.0, 'reserveUSD': 0.0, 'totalSupply': 0.0, 'txCount': 0, 'reserve00': 0.0, 'reserve11': 0.0, 'token0.id': '0xae03b12f4013ebbb5ea432cb52fc35e9598b574c', 'token0.name': 'Oracul Ai', 'token